In [457]:
# import pandas (library for dataframes)
import pandas as pd
pd.set_option('display.max_columns', None)
# import listdir (library to list directories)
from os import listdir
import numpy as np
import itertools
import warnings
warnings.filterwarnings('ignore')

In [458]:
from datetime import datetime, timedelta
import xlsxwriter
import inflect
import math

In [459]:
def breathing_calculations(file_name, file_suffix, file_location):

    # Check if file exists in file location
    file_exists = False
    for dir_file in listdir(file_location):
        if dir_file == file_name + file_suffix:
            file_exists = True

    if not file_exists:
        print("File: " + file_name + file_suffix + " does not exist in " + file_location)
        raise FileNotFoundError("File: " + file_name + file_suffix + " does not exist in " + file_location)

    # Read in data from file into dataframe
    data = None
    column_names = ["row_number","cpu_date","cpu_time","site_time","period_time","Action","title",
                "description","Blank", "Breath_Num","first_beat_id","last_beat_id", "Ti","Te",
                "PIF","PEF", "TV", "EV","RT","MV", "f", "EIP", "EEP", "Penh", "EF50", "RH", 
                "Tbox", "Tbody", "Patm", "VCF", "AV", "Sr", "n"]
    with open(file_location + file_name + file_suffix, encoding='unicode_escape') as f:
        data = pd.read_csv(f, header=None, names=column_names, skip_blank_lines=True, na_filter=True, dtype='string', skipinitialspace=True, sep='\t')

    # row numbers when Action column is period-start
    period_start_rows = data[data["Action"] == "period-start"].index

    # start the data at the row of the first period-start Action
    first_period_start = int(period_start_rows[0])
    #print(first_period_start)

    # row numbers when Action column is period-stop
    period_stop_rows = data[data["Action"] == "period-stop "].index

    # end the data at the row of the last period-stop Action
    last_period_stop = int(period_stop_rows[-1])
    #print(last_period_stop)

    # change data to be from first period-start to last period-stop
    data = data.iloc[first_period_start:last_period_stop+1]

    # Drop unnecessary columns/rows
    data = data.drop(["first_beat_id","last_beat_id"], axis="columns")
    data = data.drop("Blank", axis="columns")
    data = data.drop(["row_number", "cpu_date", "cpu_time"], axis="columns")

    analyzer_row_numbers = data[data["Action"] == "analyzer-tuning"].index
    data = data.drop(analyzer_row_numbers)

    # drop 0 rows
    data = data[(data["Ti"] != "0") | (data["title"].notna())]

    # convert necessary columns to float types
    num_cols = len(data.columns)
    list_cols_to_convert_to_float = list(range(6, num_cols))
    for i in list_cols_to_convert_to_float:
        data.iloc[:, i] = data.iloc[:, i].map(float, na_action='ignore')

    # Reset index to start from 0
    data = data.reset_index(drop=True)

    # Period start and period stop rows change after reset_index, so reassign variables
    period_start_rows = data[data["Action"] == "period-start"].index
    period_stop_rows = data[data["Action"] == "period-stop "].index

    # Number of Breaths per condition
    num_breaths_per_condition = period_stop_rows - period_start_rows - 1

    # Get title condition for each period-start Action
    conditions = list(data[data["Action"] == "period-start"]["title"].values)

    # Replace Ramp if exists
    if "Hypoxia" in conditions:
        hypoxia_indices = [i for i, x in enumerate(conditions) if x == 'Hypoxia']
        for index in hypoxia_indices:
            # Change Ramp to Ramp Up After Hypoxia (if exists)
            if conditions[index + 1] == "Ramp":
                conditions[index + 1] = "Ramp Up"

            # Change Ramp to Ramp Down Before Hypoxia (if exists)
            if conditions[index - 1] == "Ramp":
                conditions[index - 1] = "Ramp Down"

    # Loop through number of conditions to average data for each condition
    breath_baseline = None
    breath_hypercapnia = None
    breath_hypoxia_lst = []
    breath_ramp_down_lst = []
    breath_ramp_up_lst = []
    breath_ramp_lst = []
    breath_normoxia_lst = []

    baseline_df = None
    hypercapnia_df = None
    hypoxia_lst = []
    ramp_down_lst = []
    ramp_up_lst = []
    ramp_lst = []
    normoxia_lst = []

    for i in range(len(conditions)):
        condition = conditions[i]
        condition_start_row = period_start_rows[i] + 1
        condition_stop_row = period_stop_rows[i]
        condition_rows_data = data.iloc[condition_start_row:condition_stop_row, 6:]
        filtered_TV = condition_rows_data[condition_rows_data["TV"] < 0.8]
        one_averaged_data = filtered_TV.mean()
        breath_count = pd.DataFrame([num_breaths_per_condition[i]], columns = ['Breath_Count'])
        
        if condition == 'Baseline':
            baseline_df = one_averaged_data.to_frame().T
            breath_baseline = breath_count
        elif condition == 'Hypoxia':
            hypoxia_lst.append(one_averaged_data)
            breath_hypoxia_lst.append(breath_count)
        elif condition == 'Ramp Down':
            ramp_down_lst.append(one_averaged_data)
            breath_ramp_down_lst.append(breath_count)
        elif condition == 'Ramp Up':
            ramp_up_lst.append(one_averaged_data)
            breath_ramp_up_lst.append(breath_count)
        elif condition == 'Ramp':
            ramp_lst.append(one_averaged_data)
            breath_ramp_lst.append(breath_count)
        elif condition == 'Hypercapnia':
            hypercapnia_df = one_averaged_data.to_frame().T
            breath_hypercapnia = breath_count
        elif condition == 'Normoxia':
            normoxia_lst.append(one_averaged_data)
            breath_normoxia_lst.append(breath_count)

    # Append breath count to each condition
    baseline_df = breath_baseline.join(baseline_df)
    hypoxia_lst = loop_add_breath_count(breath_hypoxia_lst, hypoxia_lst)
    ramp_down_lst = loop_add_breath_count(breath_ramp_down_lst, ramp_down_lst)
    ramp_up_lst = loop_add_breath_count(breath_ramp_up_lst, ramp_up_lst)
    ramp_lst = loop_add_breath_count(breath_ramp_lst, ramp_lst)
    hypercapnia_df = breath_hypercapnia.join(hypercapnia_df)
    normoxia_lst = loop_add_breath_count(breath_normoxia_lst, normoxia_lst)

    # Normal avg calcs
    avg_hypoxia = pd.DataFrame(sum(hypoxia_lst) / len(hypoxia_lst))
    avg_ramp_down = pd.DataFrame(sum(ramp_down_lst) / len(ramp_down_lst))
    avg_ramp_up = pd.DataFrame(sum(ramp_up_lst) / len(ramp_up_lst))
    avg_ramp = pd.DataFrame(sum(ramp_lst) / len(ramp_lst))
    avg_normoxia = pd.DataFrame(sum(normoxia_lst) / len(normoxia_lst))

    # Special calcs
    avgHypoxia_avgNormoxia = avg_hypoxia / avg_normoxia
    hypoxia1_baseline = hypoxia_lst[0] / baseline_df
    hypoxia2_normoxia1 = hypoxia_lst[1] / normoxia_lst[0]
    hypercapnia_normoxia2 = hypercapnia_df / normoxia_lst[1]
    avg_N2N3 = (normoxia_lst[1] + normoxia_lst[2]) / 2
    hypercapnia_avgN2N3 = hypercapnia_df / avg_N2N3

    # Append file_dir_row to each average condition, individual condition, special conditions
    file_dir_row = file_dir_data[file_dir_data["FileName"] == file_name]

    # Drop index to allow joins to continue
    file_dir_row.reset_index(drop=True, inplace=True)

    baseline_df = file_dir_row.join(baseline_df)
    avg_hypoxia = file_dir_row.join(avg_hypoxia)
    avg_ramp_down = file_dir_row.join(avg_ramp_down)
    avg_ramp_up = file_dir_row.join(avg_ramp_up)
    avg_ramp = file_dir_row.join(avg_ramp)
    hypercapnia_df = file_dir_row.join(hypercapnia_df)
    avg_normoxia = file_dir_row.join(avg_normoxia)

    hypoxia_lst = loop_add_file_dir(file_dir_row, hypoxia_lst)
    ramp_down_lst = loop_add_file_dir(file_dir_row, ramp_down_lst)
    ramp_up_lst = loop_add_file_dir(file_dir_row, ramp_up_lst)
    ramp_lst = loop_add_file_dir(file_dir_row, ramp_lst)
    normoxia_lst = loop_add_file_dir(file_dir_row, normoxia_lst)

    avgHypoxia_avgNormoxia = file_dir_row.join(avgHypoxia_avgNormoxia)
    hypoxia1_baseline = file_dir_row.join(hypoxia1_baseline)
    hypoxia2_normoxia1 = file_dir_row.join(hypoxia2_normoxia1)
    hypercapnia_normoxia2 = file_dir_row.join(hypercapnia_normoxia2)
    avg_N2N3 = file_dir_row.join(avg_N2N3)
    hypercapnia_avgN2N3 = file_dir_row.join(hypercapnia_avgN2N3)


    # Set as global first to access global variable
    global total_baseline
    global total_avg_hypoxia
    global total_avg_ramp_down
    global total_avg_ramp_up
    global total_avg_ramp
    global total_avg_normoxia
    global total_hypoxia_1
    global total_hypoxia_2
    global total_ramp_down_1
    global total_ramp_down_2
    global total_ramp_up_1
    global total_ramp_up_2
    global total_ramp_1
    global total_ramp_2
    global total_hypercapnia
    global total_normoxia_1
    global total_normoxia_2
    global total_normoxia_3
    global total_avgHypoxia_avgNormoxia
    global total_hypoxia1_baseline
    global total_hypoxia2_normoxia1
    global total_hypercapnia_normoxia2
    global total_avg_N2N3
    global total_hypercapnia_avgN2N3

    # Append to large dataframe
    total_baseline.append(baseline_df)

    total_avg_hypoxia.append(avg_hypoxia)
    total_avg_ramp_down.append(avg_ramp_down)
    total_avg_ramp_up.append(avg_ramp_up)
    total_avg_ramp.append(avg_ramp)
    total_avg_normoxia.append(avg_normoxia)

    total_hypoxia_1.append(hypoxia_lst[0])
    total_hypoxia_2.append(hypoxia_lst[1])

    total_ramp_down_1.append(ramp_down_lst[0])
    total_ramp_down_2.append(ramp_down_lst[1])

    total_ramp_up_1.append(ramp_up_lst[0])
    total_ramp_up_2.append(ramp_up_lst[1])

    total_ramp_1.append(ramp_lst[0])
    total_ramp_2.append(ramp_lst[1])

    total_hypercapnia.append(hypercapnia_df)

    total_normoxia_1.append(normoxia_lst[0])
    total_normoxia_2.append(normoxia_lst[1])
    total_normoxia_3.append(normoxia_lst[2])

    total_avgHypoxia_avgNormoxia.append(avgHypoxia_avgNormoxia)
    total_hypoxia1_baseline.append(hypoxia1_baseline)
    total_hypoxia2_normoxia1.append(hypoxia2_normoxia1)
    total_hypercapnia_normoxia2.append(hypercapnia_normoxia2)
    total_avg_N2N3.append(avg_N2N3)
    total_hypercapnia_avgN2N3.append(hypercapnia_avgN2N3)


In [460]:
def loop_add_breath_count(breath_count_lst, lst):
    new_lst = []
    for i in range(len(lst)):
        temp_df = breath_count_lst[i].join(lst[i].to_frame().T)
        new_lst.append(temp_df)
    return new_lst

def loop_add_file_dir(file_dir, lst):
    new_lst = []
    for i in range(len(lst)):
        temp_df = lst[i]
        new_lst.append(file_dir.join(temp_df))
    return new_lst

In [461]:
def condition_mean(cond1, cond2, row_name, averaged_data):
    if cond1 in averaged_data["Conditions"].values and cond2 in averaged_data["Conditions"].values:
        cond1_data = averaged_data[averaged_data["Conditions"] == cond1].iloc[:, 1:].values[0]
        cond2_data = averaged_data[averaged_data["Conditions"] == cond2].iloc[:, 1:].values[0]
        mean = [(cond1_i + cond2_i) / 2 for cond1_i, cond2_i in zip(cond1_data, cond2_data)]
        mean.insert(0, row_name)
        averaged_data.loc[len(averaged_data.index)] = mean
        return averaged_data

In [462]:
def condition_reduction(cond1, cond2, row_name, averaged_data):
    if cond1 in averaged_data["Conditions"].values and cond2 in averaged_data["Conditions"].values:
        # Get values of condition 1 data and condition 2 data as lists
        cond1_data = averaged_data[averaged_data["Conditions"] == cond1].iloc[:, 1:].values[0]
        cond2_data = averaged_data[averaged_data["Conditions"] == cond2].iloc[:, 1:].values[0]
        reduction = list((np.subtract(cond2_data, cond1_data) / cond1_data) * 100)
        reduction.insert(0, row_name)
        averaged_data.loc[len(averaged_data.index)] = reduction
        return averaged_data

In [463]:
# data - cleaned data
# title - the title/condition 5 min bin
# averaged data - averaged analysis table
def create_5min_bins(data, title, frequency, averaged_data):
    title_index = data[data["title"] == title].index

    # Excludes the period-start, period-stop rows
    title_data_rows = data.iloc[title_index[0]+1:title_index[1], :]

    # Convert period time to allow math calculations
    title_period_time = title_data_rows["period_time"]
    lst_time = []
    for period_time in title_period_time:
        new_time = None
        try:
            new_time = datetime.strptime(period_time, '%M:%S.%f')
        except:
            new_time = datetime.strptime(period_time, '%H:%M:%S.%f')
        
        lst_time.append(new_time)

    title_data_rows.loc[:, "period_time"] = lst_time

    # Set index to allow groupby 5 min aggregations
    title_data_rows = title_data_rows.set_index("period_time")

    # Convert analysis columns to floats to allow calculations for averages
    title_cols = len(title_data_rows.columns)
    list_cols_to_convert_to_float = list(range(4, title_cols))
    for i in list_cols_to_convert_to_float:
        title_data_rows.iloc[:, i] = title_data_rows.iloc[:, i].map(float, na_action='ignore')

    # Groupby 5 minute intervals, count number of rows in each 5 minute interval to determine breath count
    title_breath_num_count = title_data_rows.groupby(pd.Grouper(freq=str(frequency) + 'T')).agg({"Breath_Num":  "count"})
    title_breath_num_count = title_breath_num_count.reset_index(drop=True)

    # Take mean of analysis columns by 5 minute intervals
    title_analysis_5m_means = title_data_rows.iloc[:, 5:].resample(str(frequency) + "T").mean()
    title_analysis_5m_means = title_analysis_5m_means.reset_index(drop=True)

    # Create condition names for each 5 min interval for title (e.g. PI_first5, PI_second5, etc.)
    title_name = None
    if title == "Post Injection":
        title_name = "PI"
    elif title == "Recovery":
        title_name = "REC"
    else:
        title_name = "Title"

    num_title_rows = len(title_analysis_5m_means.index)
    list_5min_title_condition = []
    inflect_engine = inflect.engine()
    for i in range(1, num_title_rows + 1):
        list_5min_title_condition.append(title_name + "_" + inflect_engine.number_to_words(inflect_engine.ordinal(i)) + str(frequency))
    
    title_condition_df = pd.DataFrame(list_5min_title_condition, columns=["Conditions"])
    
    # Combine condition with breath count and analysis averages dataframe
    condition_breath = title_condition_df.join(title_breath_num_count)
    condition_breath_analysis = condition_breath.join(title_analysis_5m_means)
    condition_breath_analysis = condition_breath_analysis.rename(columns={"Breath_Num": "Breath_Count"})

    # Combine new 5Min Bin Dataframe with other averaged data conditions
    averages_and_5MinAnalysis = averaged_data.append(condition_breath_analysis)
    return averages_and_5MinAnalysis

In [464]:
# data - cleaned data
# title - the title/condition 5 min bin
# averaged data - averaged analysis table
def create_own_bins(data, title, start, end, averaged_data):
    title_index = data[data["title"] == title].index

    # Excludes the period-start, period-stop rows
    title_data_rows = data.iloc[title_index[0]+1:title_index[1], :]

    # Convert period time to allow math calculations
    title_period_time = title_data_rows["period_time"]
    lst_time = []
    for period_time in title_period_time:
        new_time = None
        try:
            new_time = datetime.strptime(period_time, '%M:%S.%f')
        except:
            new_time = datetime.strptime(period_time, '%H:%M:%S.%f')
        
        lst_time.append(new_time)

    title_data_rows.loc[:, "period_time"] = lst_time

    rows_between_time = title_data_rows[(title_data_rows["period_time"].dt.minute > start-1) & (title_data_rows["period_time"].dt.minute < end)]

    num_breaths = [int(list(rows_between_time["Breath_Num"])[-1]) - int(list(rows_between_time["Breath_Num"])[0])]
    num_breaths_df = pd.DataFrame(num_breaths, columns=["Breath_Num"])

    # Convert analysis columns to floats to allow calculations for averages
    title_cols = len(rows_between_time.columns)
    list_cols_to_convert_to_float = list(range(6, title_cols))
    for i in list_cols_to_convert_to_float:
        rows_between_time.iloc[:, i] = rows_between_time.iloc[:, i].map(float, na_action='ignore')

    mean_analysis_df = rows_between_time.iloc[:, 6:].mean(axis=0)
    analysis_average_df = pd.DataFrame(mean_analysis_df.to_dict(), index=[0])

    # Create condition names for each 5 min interval for title (e.g. PI_first5, PI_second5, etc.)
    title_name = None
    if title == "Post Injection":
        title_name = "PI"
    elif title == "Recovery":
        title_name = "REC"
    else:
        title_name = "Title"
    
    lst_title_name = [title_name + "_" + str(start) + "-" + str(end)]

    title_condition_df = pd.DataFrame(lst_title_name, columns=["Conditions"])
    
    # Combine condition with breath count and analysis averages dataframe
    condition_breath = title_condition_df.join(num_breaths_df)
    condition_breath_analysis = condition_breath.join(analysis_average_df)
    condition_breath_analysis = condition_breath_analysis.rename(columns={"Breath_Num": "Breath_Count"})

    # # Combine new 5Min Bin Dataframe with other averaged data conditions
    averages_and_5MinAnalysis = averaged_data.append(condition_breath_analysis)
    return averages_and_5MinAnalysis

In [465]:
# open file directory and save data to dataframe
with open('inputs/noninjection_marcus/___FILE DIRECTORY___.csv', encoding='unicode_escape') as f:
    file_dir_data = pd.read_csv(f, header=0, skip_blank_lines=True, na_filter=True, dtype='string', skipinitialspace=True, sep=',')
file_dir_data

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,230131_chamber1_ajc7386_f03bl,1,1,MC,2023-01-31,<NA>,ajc7320,f,bl,10,filler,filler,filler,filler,filler,22,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,230131_chamber2_ajc7320_f10bl,2,1,MC,2023-01-31,<NA>,ajc7386,f,bl,3,filler,filler,filler,filler,filler,21.2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,230131_chamber2_ajc7388_f30bl,2,1,MC,2023-01-31,<NA>,ajc7388,f,bl,30,filler,filler,filler,filler,filler,19,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,230207_chamber1_ajc7142_f22bl,1,2,MC + LC,2023-02-07,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,230207_chamber2_ajc7320_f10bl,2,2,MC + LC,2023-02-07,<NA>,ajc7320,f,bl,10,filler,filler,filler,filler,filler,22.8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,230207_chamber1_ajc7386_f03bl,1,2,MC + LC,2023-02-07,<NA>,ajc7386,f,bl,3,filler,filler,filler,filler,filler,20.7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,230207_chamber2_ajc7388_f30bl,2,2,MC + LC,2023-02-07,<NA>,ajc7388,f,bl,30,filler,filler,filler,filler,filler,18.7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,230221_chamber1_ajc6660_m03bl,1,1,TL,2023-02-21,<NA>,ajc6660,m,bl,3,filler,filler,filler,filler,filler,32.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,230221_chamber2_ajc6662_m22bl,2,1,TL,2023-02-21,<NA>,ajc6662,m,bl,22,filler,filler,filler,filler,filler,29,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [466]:
# # ------------------------------------ TESTING CELL -------------------------------------

# file_name = "230131_chamber1_ajc7142_f22bl"
# file_suffix = ".rf_1.iox.txt"
# file_location = "inputs/noninjection_marcus/"

# # Check if file exists in file location
# file_exists = False
# for dir_file in listdir(file_location):
#     if dir_file == file_name + file_suffix:
#         file_exists = True

# if not file_exists:
#     print("File: " + file_name + file_suffix + " does not exist in " + file_location)
#     raise FileNotFoundError("File: " + file_name + file_suffix + " does not exist in " + file_location)

# # Read in data from file into dataframe
# data = None
# column_names = ["row_number","cpu_date","cpu_time","site_time","period_time","Action","title",
#             "description","Blank", "Breath_Num","first_beat_id","last_beat_id", "Ti","Te",
#             "PIF","PEF", "TV", "EV","RT","MV", "f", "EIP", "EEP", "Penh", "EF50", "RH", 
#             "Tbox", "Tbody", "Patm", "VCF", "AV", "Sr", "n"]
# with open(file_location + file_name + file_suffix, encoding='unicode_escape') as f:
#     data = pd.read_csv(f, header=None, names=column_names, skip_blank_lines=True, na_filter=True, dtype='string', skipinitialspace=True, sep='\t')

# # row numbers when Action column is period-start
# period_start_rows = data[data["Action"] == "period-start"].index

# # start the data at the row of the first period-start Action
# first_period_start = int(period_start_rows[0])
# #print(first_period_start)

# # row numbers when Action column is period-stop
# period_stop_rows = data[data["Action"] == "period-stop "].index

# # end the data at the row of the last period-stop Action
# last_period_stop = int(period_stop_rows[-1])
# #print(last_period_stop)

# # change data to be from first period-start to last period-stop
# data = data.iloc[first_period_start:last_period_stop+1]

# # Drop unnecessary columns/rows
# data = data.drop(["first_beat_id","last_beat_id"], axis="columns")
# data = data.drop("Blank", axis="columns")
# data = data.drop(["row_number", "cpu_date", "cpu_time"], axis="columns")

# analyzer_row_numbers = data[data["Action"] == "analyzer-tuning"].index
# data = data.drop(analyzer_row_numbers)

# # drop 0 rows
# data = data[(data["Ti"] != "0") | (data["title"].notna())]

# # convert necessary columns to float types
# num_cols = len(data.columns)
# list_cols_to_convert_to_float = list(range(6, num_cols))
# for i in list_cols_to_convert_to_float:
#     data.iloc[:, i] = data.iloc[:, i].map(float, na_action='ignore')

# # Reset index to start from 0
# data = data.reset_index(drop=True)

# # Period start and period stop rows change after reset_index, so reassign variables
# period_start_rows = data[data["Action"] == "period-start"].index
# period_stop_rows = data[data["Action"] == "period-stop "].index

# # Number of Breaths per condition
# num_breaths_per_condition = period_stop_rows - period_start_rows - 1

# # Get title condition for each period-start Action
# conditions = list(data[data["Action"] == "period-start"]["title"].values)

# # Replace Ramp if exists
# if "Hypoxia" in conditions:
#     hypoxia_indices = [i for i, x in enumerate(conditions) if x == 'Hypoxia']
#     for index in hypoxia_indices:
#         # Change Ramp to Ramp Up After Hypoxia (if exists)
#         if conditions[index + 1] == "Ramp":
#             conditions[index + 1] = "Ramp Up"

#         # Change Ramp to Ramp Down Before Hypoxia (if exists)
#         if conditions[index - 1] == "Ramp":
#             conditions[index - 1] = "Ramp Down"

# # Loop through number of conditions to average data for each condition
# breath_baseline = None
# breath_hypercapnia = None
# breath_hypoxia_lst = []
# breath_ramp_down_lst = []
# breath_ramp_up_lst = []
# breath_ramp_lst = []
# breath_normoxia_lst = []

# baseline_df = None
# hypercapnia_df = None
# hypoxia_lst = []
# ramp_down_lst = []
# ramp_up_lst = []
# ramp_lst = []
# normoxia_lst = []

# for i in range(len(conditions)):
#     condition = conditions[i]
#     condition_start_row = period_start_rows[i] + 1
#     condition_stop_row = period_stop_rows[i]
#     condition_rows_data = data.iloc[condition_start_row:condition_stop_row, 6:]
#     filtered_TV = condition_rows_data[condition_rows_data["TV"] < 0.8]
#     one_averaged_data = filtered_TV.mean()
#     breath_count = pd.DataFrame([num_breaths_per_condition[i]], columns = ['Breath_Count'])
    
#     if condition == 'Baseline':
#         baseline_df = one_averaged_data.to_frame().T
#         breath_baseline = breath_count
#     elif condition == 'Hypoxia':
#         hypoxia_lst.append(one_averaged_data)
#         breath_hypoxia_lst.append(breath_count)
#     elif condition == 'Ramp Down':
#         ramp_down_lst.append(one_averaged_data)
#         breath_ramp_down_lst.append(breath_count)
#     elif condition == 'Ramp Up':
#         ramp_up_lst.append(one_averaged_data)
#         breath_ramp_up_lst.append(breath_count)
#     elif condition == 'Ramp':
#         ramp_lst.append(one_averaged_data)
#         breath_ramp_lst.append(breath_count)
#     elif condition == 'Hypercapnia':
#         hypercapnia_df = one_averaged_data.to_frame().T
#         breath_hypercapnia = breath_count
#     elif condition == 'Normoxia':
#         normoxia_lst.append(one_averaged_data)
#         breath_normoxia_lst.append(breath_count)

# # Append breath count to each condition
# baseline_df = breath_baseline.join(baseline_df)
# hypoxia_lst = loop_add_breath_count(breath_hypoxia_lst, hypoxia_lst)
# ramp_down_lst = loop_add_breath_count(breath_ramp_down_lst, ramp_down_lst)
# ramp_up_lst = loop_add_breath_count(breath_ramp_up_lst, ramp_up_lst)
# ramp_lst = loop_add_breath_count(breath_ramp_lst, ramp_lst)
# hypercapnia_df = breath_hypercapnia.join(hypercapnia_df)
# normoxia_lst = loop_add_breath_count(breath_normoxia_lst, normoxia_lst)

# # Normal avg calcs
# avg_hypoxia = pd.DataFrame(sum(hypoxia_lst) / len(hypoxia_lst))
# avg_ramp_down = pd.DataFrame(sum(ramp_down_lst) / len(ramp_down_lst))
# avg_ramp_up = pd.DataFrame(sum(ramp_up_lst) / len(ramp_up_lst))
# avg_ramp = pd.DataFrame(sum(ramp_lst) / len(ramp_lst))
# avg_normoxia = pd.DataFrame(sum(normoxia_lst) / len(normoxia_lst))

# # Special calcs
# avgHypoxia_avgNormoxia = avg_hypoxia / avg_normoxia
# hypoxia1_baseline = hypoxia_lst[0] / baseline_df
# hypoxia2_normoxia1 = hypoxia_lst[1] / normoxia_lst[0]
# hypercapnia_normoxia2 = hypercapnia_df / normoxia_lst[1]
# avg_N2N3 = (normoxia_lst[1] + normoxia_lst[2]) / 2
# hypercapnia_avgN2N3 = hypercapnia_df / avg_N2N3

# # Append file_dir_row to each average condition, individual condition, special conditions
# file_dir_row = file_dir_data[file_dir_data["FileName"] == file_name]

# baseline_df = file_dir_row.join(baseline_df)
# avg_hypoxia = file_dir_row.join(avg_hypoxia)
# avg_ramp_down = file_dir_row.join(avg_ramp_down)
# avg_ramp_up = file_dir_row.join(avg_ramp_up)
# avg_ramp = file_dir_row.join(avg_ramp)
# hypercapnia_df = file_dir_row.join(hypercapnia_df)
# avg_normoxia = file_dir_row.join(avg_normoxia)

# hypoxia_lst = loop_add_file_dir(file_dir_row, hypoxia_lst)
# ramp_down_lst = loop_add_file_dir(file_dir_row, ramp_down_lst)
# ramp_up_lst = loop_add_file_dir(file_dir_row, ramp_up_lst)
# ramp_lst = loop_add_file_dir(file_dir_row, ramp_lst)
# normoxia_lst = loop_add_file_dir(file_dir_row, normoxia_lst)

# avgHypoxia_avgNormoxia = file_dir_row.join(avgHypoxia_avgNormoxia)
# hypoxia1_baseline = file_dir_row.join(hypoxia1_baseline)
# hypoxia2_normoxia1 = file_dir_row.join(hypoxia2_normoxia1)
# hypercapnia_normoxia2 = file_dir_row.join(hypercapnia_normoxia2)
# avg_N2N3 = file_dir_row.join(avg_N2N3)
# hypercapnia_avgN2N3 = file_dir_row.join(hypercapnia_avgN2N3)


# # Set as global first to access global variable
# global total_baseline
# global total_avg_hypoxia
# global total_avg_ramp_down
# global total_avg_ramp_up
# global total_avg_ramp
# global total_avg_normoxia
# global total_hypoxia_1
# global total_hypoxia_2
# global total_ramp_down_1
# global total_ramp_down_2
# global total_ramp_up_1
# global total_ramp_up_2
# global total_ramp_1
# global total_ramp_2
# global total_hypercapnia
# global total_normoxia_1
# global total_normoxia_2
# global total_normoxia_3
# global total_avgHypoxia_avgNormoxia
# global total_hypoxia1_baseline
# global total_hypoxia2_normoxia1
# global total_hypercapnia_normoxia2
# global total_avg_N2N3
# global total_hypercapnia_avgN2N3

# # Append to large dataframe
# total_baseline.append(baseline_df)

# total_avg_hypoxia.append(avg_hypoxia)
# total_avg_ramp_down.append(avg_ramp_down)
# total_avg_ramp_up.append(avg_ramp_up)
# total_avg_ramp.append(avg_ramp)
# total_avg_normoxia.append(avg_normoxia)

# total_hypoxia_1.append(hypoxia_lst[0])
# total_hypoxia_2.append(hypoxia_lst[1])

# total_ramp_down_1.append(ramp_down_lst[0])
# total_ramp_down_2.append(ramp_down_lst[1])

# total_ramp_up_1.append(ramp_up_lst[0])
# total_ramp_up_2.append(ramp_up_lst[1])

# total_ramp_1.append(ramp_lst[0])
# total_ramp_2.append(ramp_lst[1])

# total_hypercapnia.append(hypercapnia_df)

# total_normoxia_1.append(normoxia_lst[0])
# total_normoxia_2.append(normoxia_lst[1])
# total_normoxia_3.append(normoxia_lst[2])

# total_avgHypoxia_avgNormoxia.append(avgHypoxia_avgNormoxia)
# total_hypoxia1_baseline.append(hypoxia1_baseline)
# total_hypoxia2_normoxia1.append(hypoxia2_normoxia1)
# total_hypercapnia_normoxia2.append(hypercapnia_normoxia2)
# total_avg_N2N3.append(avg_N2N3)
# total_hypercapnia_avgN2N3.append(hypercapnia_avgN2N3)


In [467]:
# Convert weight column to float for calculations
file_dir_data["Weight"] = file_dir_data["Weight"].map(float, na_action=None)

# Convert Date_Tested column to datetime for calculations
file_dir_data["Date_Tested"] = file_dir_data["Date_Tested"].map(lambda date: datetime.strptime(date, "%Y-%m-%d"))

# Convert DOB column to datetime for calculations
# file_dir_data["DOB"] = file_dir_data["DOB"].map(lambda date: datetime.datetime.strptime(date, "%Y-%m-%d"))

In [468]:
# Add Age column
# file_dir_data["Age"] = file_dir_data["Date_Tested"] - file_dir_data["DOB"]
#file_dir_data["Age"] = fillna()

In [469]:
# Retrieve file list from file directory
file_list = file_dir_data["FileName"]
file_list

0       230131_chamber1_ajc7142_f22bl
1       230131_chamber1_ajc7386_f03bl
2       230131_chamber2_ajc7320_f10bl
3       230131_chamber2_ajc7388_f30bl
4       230207_chamber1_ajc7142_f22bl
5       230207_chamber2_ajc7320_f10bl
6       230207_chamber1_ajc7386_f03bl
7       230207_chamber2_ajc7388_f30bl
8       230221_chamber1_ajc6660_m03bl
9       230221_chamber2_ajc6662_m22bl
10      230221_chamber1_ajc7137_m10bl
11      230221_chamber2_ajc7138_m30bl
12      230228_chamber1_ajc6660_m03bl
13      230228_chamber2_ajc6662_m22bl
14      230228_chamber1_ajc7137_m10bl
15      230228_chamber2_ajc7138_m30bl
16      230426_chamber1_ajc7569_m22bl
17      230426_chamber2_ajc7572_m20bl
18      230426_chamber1_ajc7729_m01bl
19      230426_chamber2_ajc7730_m02bl
20      230502_chamber2_ajc7615_f20bl
21      230502_chamber2_ajc7620_f22bl
22      230602_chamber1_ajc7634_m02bl
23      230602_chamber2_ajc7635_m03bl
24      230602_chamber1_ajc7753_m20bl
25      230602_chamber2_ajc7754_m30bl
26      2306

In [470]:
total_baseline = []

total_avg_hypoxia = []
total_avg_ramp_down = []
total_avg_ramp_up = []
total_avg_ramp = []
total_avg_normoxia = []

total_hypoxia_1 = []
total_hypoxia_2 = []

total_ramp_down_1 = []
total_ramp_down_2 = []

total_ramp_up_1 = []
total_ramp_up_2 = []

total_ramp_1 = []
total_ramp_2 = []

total_hypercapnia = []

total_normoxia_1 = []
total_normoxia_2 = []
total_normoxia_3 = []

total_avgHypoxia_avgNormoxia = []
total_hypoxia1_baseline = []
total_hypoxia2_normoxia1 = []
total_hypercapnia_normoxia2 = []
total_avg_N2N3 = []
total_hypercapnia_avgN2N3 = []

In [471]:
# Apply breathing_calculations to each file
file_location = "inputs/noninjection_marcus/"
file_suffix = ".rf_1.iox.txt"
for file_name in file_list:
    print(file_name)
    breathing_calculations(file_name, file_suffix, file_location)
    
print("Done")

230131_chamber1_ajc7142_f22bl


230131_chamber1_ajc7386_f03bl
230131_chamber2_ajc7320_f10bl
230131_chamber2_ajc7388_f30bl
230207_chamber1_ajc7142_f22bl
230207_chamber2_ajc7320_f10bl
230207_chamber1_ajc7386_f03bl
230207_chamber2_ajc7388_f30bl
230221_chamber1_ajc6660_m03bl
230221_chamber2_ajc6662_m22bl
230221_chamber1_ajc7137_m10bl
230221_chamber2_ajc7138_m30bl
230228_chamber1_ajc6660_m03bl
230228_chamber2_ajc6662_m22bl
230228_chamber1_ajc7137_m10bl
230228_chamber2_ajc7138_m30bl
230426_chamber1_ajc7569_m22bl
230426_chamber2_ajc7572_m20bl
230426_chamber1_ajc7729_m01bl
230426_chamber2_ajc7730_m02bl
230502_chamber2_ajc7615_f20bl
230502_chamber2_ajc7620_f22bl
230602_chamber1_ajc7634_m02bl
230602_chamber2_ajc7635_m03bl
230602_chamber1_ajc7753_m20bl
230602_chamber2_ajc7754_m30bl
230602_chamber1_ajc7931_m03bl
230602_chamber2_ajc7934_m30bl
230609_chamber1_ajc7620_f22bl_1
230609_chamber2_ajc7569_m22bl_1
230609_chamber1_ajc7931_m03bl
230609_chamber2_ajc7753_m20bl
230610_chamber1_ajc7729_m01bl
230610_chamber2_ajc7572_m20bl
230610

In [472]:
# Convert list to dataframes
total_baseline = pd.concat(total_baseline, ignore_index=True)

total_avg_hypoxia = pd.concat(total_avg_hypoxia, ignore_index=True)
total_avg_ramp_down = pd.concat(total_avg_ramp_down, ignore_index=True)
total_avg_ramp_up = pd.concat(total_avg_ramp_up, ignore_index=True)
total_avg_ramp = pd.concat(total_avg_ramp, ignore_index=True)
total_avg_normoxia = pd.concat(total_avg_normoxia, ignore_index=True)

total_hypoxia_1 = pd.concat(total_hypoxia_1, ignore_index=True)
total_hypoxia_2 = pd.concat(total_hypoxia_2, ignore_index=True)

total_ramp_down_1 = pd.concat(total_ramp_down_1, ignore_index=True)
total_ramp_down_2 = pd.concat(total_ramp_down_2, ignore_index=True)

total_ramp_up_1 = pd.concat(total_ramp_up_1, ignore_index=True)
total_ramp_up_2 = pd.concat(total_ramp_up_2, ignore_index=True)

total_ramp_1 = pd.concat(total_ramp_1, ignore_index=True)
total_ramp_2 = pd.concat(total_ramp_2, ignore_index=True)

total_hypercapnia = pd.concat(total_hypercapnia, ignore_index=True)

total_normoxia_1 = pd.concat(total_normoxia_1, ignore_index=True)
total_normoxia_2 = pd.concat(total_normoxia_2, ignore_index=True)
total_normoxia_3 = pd.concat(total_normoxia_3, ignore_index=True)

total_avgHypoxia_avgNormoxia = pd.concat(total_avgHypoxia_avgNormoxia, ignore_index=True)
total_hypoxia1_baseline = pd.concat(total_hypoxia1_baseline, ignore_index=True)
total_hypoxia2_normoxia1 = pd.concat(total_hypoxia2_normoxia1, ignore_index=True)
total_hypercapnia_normoxia2 = pd.concat(total_hypercapnia_normoxia2, ignore_index=True)
total_avg_N2N3 = pd.concat(total_avg_N2N3, ignore_index=True)
total_hypercapnia_avgN2N3 = pd.concat(total_hypercapnia_avgN2N3, ignore_index=True)


In [473]:
# Normalize MV, TV by weight
total_baseline["MV/g"] = total_baseline["MV"] / total_baseline["Weight"]
total_baseline["TV/g"] = total_baseline["TV"] / total_baseline["Weight"]

total_avg_hypoxia["MV/g"] = total_avg_hypoxia["MV"] / total_avg_hypoxia["Weight"]
total_avg_hypoxia["TV/g"] = total_avg_hypoxia["TV"] / total_avg_hypoxia["Weight"]

total_avg_ramp_down["MV/g"] = total_avg_ramp_down["MV"] / total_avg_ramp_down["Weight"]
total_avg_ramp_down["TV/g"] = total_avg_ramp_down["TV"] / total_avg_ramp_down["Weight"]

total_avg_ramp_up["MV/g"] = total_avg_ramp_up["MV"] / total_avg_ramp_up["Weight"]
total_avg_ramp_up["TV/g"] = total_avg_ramp_up["TV"] / total_avg_ramp_up["Weight"]

total_avg_ramp["MV/g"] = total_avg_ramp["MV"] / total_avg_ramp["Weight"]
total_avg_ramp["TV/g"] = total_avg_ramp["TV"] / total_avg_ramp["Weight"]

total_avg_normoxia["MV/g"] = total_avg_normoxia["MV"] / total_avg_normoxia["Weight"]
total_avg_normoxia["TV/g"] = total_avg_normoxia["TV"] / total_avg_normoxia["Weight"]

total_hypoxia_1["MV/g"] = total_hypoxia_1["MV"] / total_hypoxia_1["Weight"]
total_hypoxia_1["TV/g"] = total_hypoxia_1["TV"] / total_hypoxia_1["Weight"]

total_hypoxia_2["MV/g"] = total_hypoxia_2["MV"] / total_hypoxia_2["Weight"]
total_hypoxia_2["TV/g"] = total_hypoxia_2["TV"] / total_hypoxia_2["Weight"]

total_ramp_down_1["MV/g"] = total_ramp_down_1["MV"] / total_ramp_down_1["Weight"]
total_ramp_down_1["TV/g"] = total_ramp_down_1["TV"] / total_ramp_down_1["Weight"]

total_ramp_down_2["MV/g"] = total_ramp_down_2["MV"] / total_ramp_down_2["Weight"]
total_ramp_down_2["TV/g"] = total_ramp_down_2["TV"] / total_ramp_down_2["Weight"]

total_ramp_up_1["MV/g"] = total_ramp_up_1["MV"] / total_ramp_up_1["Weight"]
total_ramp_up_1["TV/g"] = total_ramp_up_1["TV"] / total_ramp_up_1["Weight"]

total_ramp_up_2["MV/g"] = total_ramp_up_2["MV"] / total_ramp_up_2["Weight"]
total_ramp_up_2["TV/g"] = total_ramp_up_2["TV"] / total_ramp_up_2["Weight"]

total_ramp_1["MV/g"] = total_ramp_1["MV"] / total_ramp_1["Weight"]
total_ramp_1["TV/g"] = total_ramp_1["TV"] / total_ramp_1["Weight"]

total_ramp_2["MV/g"] = total_ramp_2["MV"] / total_ramp_2["Weight"]
total_ramp_2["TV/g"] = total_ramp_2["TV"] / total_ramp_2["Weight"]

total_hypercapnia["MV/g"] = total_hypercapnia["MV"] / total_hypercapnia["Weight"]
total_hypercapnia["TV/g"] = total_hypercapnia["TV"] / total_hypercapnia["Weight"]

total_normoxia_1["MV/g"] = total_normoxia_1["MV"] / total_normoxia_1["Weight"]
total_normoxia_1["TV/g"] = total_normoxia_1["TV"] / total_normoxia_1["Weight"]

total_normoxia_2["MV/g"] = total_normoxia_2["MV"] / total_normoxia_2["Weight"]
total_normoxia_2["TV/g"] = total_normoxia_2["TV"] / total_normoxia_2["Weight"]

total_normoxia_3["MV/g"] = total_normoxia_3["MV"] / total_normoxia_3["Weight"]
total_normoxia_3["TV/g"] = total_normoxia_3["TV"] / total_normoxia_3["Weight"]

total_avgHypoxia_avgNormoxia["MV/g"] = total_avgHypoxia_avgNormoxia["MV"] / total_avgHypoxia_avgNormoxia["Weight"]
total_avgHypoxia_avgNormoxia["TV/g"] = total_avgHypoxia_avgNormoxia["TV"] / total_avgHypoxia_avgNormoxia["Weight"]

total_hypoxia1_baseline["MV/g"] = total_hypoxia1_baseline["MV"] / total_hypoxia1_baseline["Weight"]
total_hypoxia1_baseline["TV/g"] = total_hypoxia1_baseline["TV"] / total_hypoxia1_baseline["Weight"]

total_hypoxia2_normoxia1["MV/g"] = total_hypoxia2_normoxia1["MV"] / total_hypoxia2_normoxia1["Weight"]
total_hypoxia2_normoxia1["TV/g"] = total_hypoxia2_normoxia1["TV"] / total_hypoxia2_normoxia1["Weight"]

total_hypercapnia_normoxia2["MV/g"] = total_hypercapnia_normoxia2["MV"] / total_hypercapnia_normoxia2["Weight"]
total_hypercapnia_normoxia2["TV/g"] = total_hypercapnia_normoxia2["TV"] / total_hypercapnia_normoxia2["Weight"]

total_avg_N2N3["MV/g"] = total_avg_N2N3["MV"] / total_avg_N2N3["Weight"]
total_avg_N2N3["TV/g"] = total_avg_N2N3["TV"] / total_avg_N2N3["Weight"]

total_hypercapnia_avgN2N3["MV/g"] = total_hypercapnia_avgN2N3["MV"] / total_hypercapnia_avgN2N3["Weight"]
total_hypercapnia_avgN2N3["TV/g"] = total_hypercapnia_avgN2N3["TV"] / total_hypercapnia_avgN2N3["Weight"]



In [474]:
# Prioritize MV, TV, f columns by rearranging column order
# Sort by Mouse_ID and then Test_Number
def prioritize_columns_and_sort(df):
    cols_to_move = ['MV', 'TV', 'f', 'MV/g', 'TV/g']
    col_to_insert_after = 'Breath_Count'

    cols = df.columns.tolist()
    for col in cols_to_move[::-1]:  # Reverse list for right popping
        cols.insert(cols.index(col_to_insert_after) + 1, cols.pop(cols.index(col)))

    df = df[cols]

    # Sort to make easier to read
    df = df.sort_values(by=['Mouse_ID', 'Test_Number'])
    return df

total_baseline = prioritize_columns_and_sort(total_baseline)

total_avg_hypoxia = prioritize_columns_and_sort(total_avg_hypoxia)
total_avg_ramp_down = prioritize_columns_and_sort(total_avg_ramp_down)
total_avg_ramp_up = prioritize_columns_and_sort(total_avg_ramp_up)
total_avg_ramp = prioritize_columns_and_sort(total_avg_ramp)
total_avg_normoxia = prioritize_columns_and_sort(total_avg_normoxia)

total_hypoxia_1 = prioritize_columns_and_sort(total_hypoxia_1)
total_hypoxia_2 = prioritize_columns_and_sort(total_hypoxia_2)

total_ramp_down_1 = prioritize_columns_and_sort(total_ramp_down_1)
total_ramp_down_2 = prioritize_columns_and_sort(total_ramp_down_2)

total_ramp_up_1 = prioritize_columns_and_sort(total_ramp_up_1)
total_ramp_up_2 = prioritize_columns_and_sort(total_ramp_up_2)

total_ramp_1 = prioritize_columns_and_sort(total_ramp_1)
total_ramp_2 = prioritize_columns_and_sort(total_ramp_2)

total_hypercapnia = prioritize_columns_and_sort(total_hypercapnia)

total_normoxia_1 = prioritize_columns_and_sort(total_normoxia_1)
total_normoxia_2 = prioritize_columns_and_sort(total_normoxia_2)
total_normoxia_3 = prioritize_columns_and_sort(total_normoxia_3)

total_avgHypoxia_avgNormoxia = prioritize_columns_and_sort(total_avgHypoxia_avgNormoxia)
total_hypoxia1_baseline = prioritize_columns_and_sort(total_hypoxia1_baseline)
total_hypoxia2_normoxia1 = prioritize_columns_and_sort(total_hypoxia2_normoxia1)
total_hypercapnia_normoxia2 = prioritize_columns_and_sort(total_hypercapnia_normoxia2)
total_avg_N2N3 = prioritize_columns_and_sort(total_avg_N2N3)
total_hypercapnia_avgN2N3 = prioritize_columns_and_sort(total_hypercapnia_avgN2N3)
total_hypercapnia_avgN2N3

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Breath_Count,MV,TV,f,MV/g,TV/g,Ti,Te,PIF,PEF,EV,RT,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n
8,230221_chamber1_ajc6660_m03bl,1,1,TL,2023-02-21,<NA>,ajc6660,m,bl,3,filler,filler,filler,filler,filler,32.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3.813505,1.655533,1.353098,1.219105,0.050939,0.041634,1.135385,0.625969,1.042891,1.896633,1.379147,0.552337,0.766038,0.779977,1.987940,1.763131,1.017537,1.000675,1.0,1.000000,1.002230,5.298215,1.611692,1.0
12,230228_chamber1_ajc6660_m03bl,1,2,MC + GK,2023-02-28,<NA>,ajc6660,m,bl,3,filler,filler,filler,filler,filler,34.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.014133,2.249232,1.567651,1.450113,0.065195,0.045439,0.894557,0.566733,1.541350,2.493948,1.573561,0.529816,1.000626,0.570438,1.746153,2.333947,0.983902,1.000630,1.0,1.000000,0.998565,1.624846,1.019484,1.0
9,230221_chamber2_ajc6662_m22bl,2,1,TL,2023-02-21,<NA>,ajc6662,m,bl,22,filler,filler,filler,filler,filler,29.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.509913,2.184927,1.477649,1.477280,0.075342,0.050953,1.068453,0.510077,1.206819,2.305853,1.504544,0.464054,1.009709,0.535996,2.104599,2.565893,0.978912,1.000393,1.0,1.000000,0.999035,2.267314,1.286095,1.0
13,230228_chamber2_ajc6662_m22bl,2,2,MC + GK,2023-02-28,<NA>,ajc6662,m,bl,22,filler,filler,filler,filler,filler,31.6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.426165,2.144287,1.459642,1.480199,0.067857,0.046191,0.976672,0.515647,1.325822,2.309159,1.493153,0.417130,1.031063,0.657542,2.623677,2.910676,1.050255,1.006487,1.0,1.000000,1.008141,2.041420,1.140606,1.0
10,230221_chamber1_ajc7137_m10bl,1,1,TL,2023-02-21,<NA>,ajc7137,m,bl,10,filler,filler,filler,filler,filler,33.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.777411,2.218585,1.547746,1.441723,0.066226,0.046201,1.009562,0.538354,1.281143,2.253851,1.547889,0.480244,1.095854,0.581725,2.149540,2.661080,1.003566,1.000949,1.0,1.000000,1.000915,1.188377,1.012430,1.0
14,230228_chamber1_ajc7137_m10bl,1,2,MC + GK,2023-02-28,<NA>,ajc7137,m,bl,10,filler,filler,filler,filler,filler,34.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.018650,2.263387,1.668141,1.351764,0.065605,0.048352,1.037011,0.553867,1.359737,2.106513,1.677005,0.529689,0.969043,0.493720,1.460386,2.518774,0.997352,1.001062,1.0,1.000000,1.000789,1.663966,1.119310,1.0
11,230221_chamber2_ajc7138_m30bl,2,1,TL,2023-02-21,<NA>,ajc7138,m,bl,30,filler,filler,filler,filler,filler,27.3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.391037,2.403793,1.464344,1.645714,0.088051,0.053639,0.979754,0.442925,1.303640,2.276406,1.454134,0.387784,0.981635,0.562558,2.027517,2.597449,0.971713,1.000025,1.0,1.000000,0.998296,2.058377,1.101904,1.0
15,230228_chamber2_ajc7138_m30bl,2,2,MC + GK,2023-02-28,<NA>,ajc7138,m,bl,30,filler,filler,filler,filler,filler,28.6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.070041,2.179790,1.425506,1.536717,0.076216,0.049843,0.911518,0.507818,1.324892,1.994288,1.427932,0.446385,0.940398,0.635381,1.894679,2.627646,0.995755,0.996069,1.0,1.000000,0.997822,1.459730,1.056815,1.0
0,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2.223787,2.011485,1.373009,1.471765,0.089798,0.061295,1.023529,0.516285,1.160005,2.072675,1.386751,0.529947,1.027196,0.474851,1.590500,2.305625,1.000000,1.000000,1.0,1.000000,1.000000,0.996902,1.575844,1.0
4,230207_chamber1_ajc7142_f22bl,1,2,MC + LC,2023-02-07,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.053056,2.173797,1.450093,1.496035,0.096186,0.064163,0.945532,0.520951,1.311020,2.394785,1.507919,0.492816,1.028910,0.557706,1.937883,2.852996,1.000000,1.000000,1.0,1.000000,1.000000,1.034556,1.348955,1.0


In [475]:
# Normalize MV and TV columns for Hypoxia/Normoxia
# TODO: WHAT TO DO WITH THIS???
# normalized = pd.DataFrame()
# normalized["MV/g"] = (master_data[master_data["Conditions"] == "Hypoxia"]["MV/g"] - master_data[master_data["Conditions"] == "Normoxia"]["MV/g"]) / master_data[master_data["Conditions"] == "Hypoxia"]["MV"]
# normalized["TV/g"] = (master_data[master_data["Conditions"] == "Hypoxia"]["TV/g"] - master_data[master_data["Conditions"] == "Normoxia"]["TV/g"]) / master_data[master_data["Conditions"] == "Hypoxia"]["TV"]
# normalized

In [476]:
# Get the averages of all protocols for each condition and test_number
# mean_of_conditions_test_number = master_data.groupby(["Conditions", "Test_Number"]).agg(np.mean).reset_index()
# master_data = master_data.append(mean_of_conditions_test_number)

In [477]:
#Get the averages of all protocols for each condition and test_number

In [478]:
# Function to highlight every Test_Number == 1
def highlight_every_number_1(s):
    df = s.copy()
    #set by condition
    mask = (df['Test_Number'] == "1") & (df['Test_Number'].notna())
    df.loc[mask, :] = 'background-color: LightGreen'
    df.loc[~mask,:] = 'color: SlateGrey'
    return df   

In [479]:
# condition_list = list(set(master_data["Conditions"].values))
# condition_list = [str(condition) for condition in condition_list] # In case of non-string values
# print(condition_list)
# condition_list.sort()

In [480]:
sheet_order = [total_avg_normoxia, total_avg_hypoxia, total_baseline, total_hypoxia_1, total_normoxia_1, total_hypoxia_2,
    total_normoxia_2, total_hypercapnia, total_normoxia_3, total_avgHypoxia_avgNormoxia, total_hypoxia1_baseline,
    total_hypoxia2_normoxia1, total_hypercapnia_normoxia2, total_hypercapnia_avgN2N3, total_avg_ramp_down, total_avg_ramp_up,
    total_avg_N2N3, total_ramp_1, total_ramp_2, total_ramp_down_1, total_ramp_down_2, total_ramp_up_1, total_ramp_up_2
]

sheet_names = ['Avg Normoxia', 'Avg Hypoxia', 'Baseline', 'Hypoxia 1', 'Normoxia 1', 'Hypoxia 2', 'Normoxia 2', 'Hypercapnia',
    'Normoxia 3', 'Avg Hypoxia-Avg Normoxia', 'Hypoxia 1-Baseline', 'Hypoxia 2-Normoxia 1', 'Hypercapnia-Normoxia 2',
    'Hypercapnia-Avg_N2N3', 'Avg Ramp Down', 'Avg Ramp Up', 'Avg_N2N3', 'Ramp 1', 'Ramp 2', 'Ramp Down 1', 'Ramp Down 2',
    'Ramp Up 1', 'Ramp Up 2'
]

writer = pd.ExcelWriter('behavioural_breathing_calcs_output.xlsx', engine='xlsxwriter')
for i in range(len(sheet_order)):
    condition_dataframe = sheet_order[i].reset_index(drop=True)
    condition_dataframe.style.apply(highlight_every_number_1, axis=None).to_excel(writer, sheet_name=str(sheet_names[i]))
writer.close()
print("All Done")

All Done
